In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import torch
from csv import DictReader
from scipy.stats import multivariate_normal as mv_norm

from MaxCUT.IsingRBM import CutRBM
%matplotlib inline

In [2]:
def hamiltonian_mc(init_func, num_samples, 
                   U, dU, K=lambda p: 0.5 * (p.T @ p), dK=lambda p: p.T,
                   L=20, delta=0.3, step=1):
    result = np.array([init_func()], dtype=np.float64)
    dim = len(result[0])
    acceptance = 0
    for x in range(num_samples):
        start_pos = np.copy(result[-1])
        start_mom = stats.multivariate_normal.rvs(cov=np.eye(dim))  # TODO?
        proposed_pos = np.copy(start_pos)
        proposed_mom = np.copy(start_mom)

        proposed_mom -= (delta / 2) * dU(start_pos)  # Half Step
        proposed_pos += delta * dK(proposed_mom)      # Whole Step
        for _ in range(L):
            proposed_mom -= delta * dU(proposed_pos)  # 2 Halfs -> Whole
            proposed_pos += delta * dK(proposed_mom)  # Always Whole
        proposed_mom -= (delta / 2) * dU(proposed_pos)
#         print(start_pos, proposed_pos)
        alpha = np.exp((U(start_pos) + K(start_mom)) - (U(proposed_pos) + K(proposed_mom))) # Acceptance
        acceptance += alpha
#         print(alpha, proposed_pos)
        next_state = proposed_pos if stats.bernoulli.rvs(min(1, alpha), size=1) else start_pos
        result = np.append(result, [next_state], axis=0)
    print(acceptance / num_samples)
    return result[::step]

def mala(init_func, num_samples, X, dX, delta=1.2, step=1):
    K = lambda q: 0.5 * (q.T @ q)
    result = np.array([init_func()], dtype=np.float64)
    dim, a = len(result[0]), 0
    for x in range(num_samples):
        start_pos = np.copy(result[-1])
        unscaled_brownian = mv_norm.rvs(cov=np.eye(dim))
        proposed = start_pos - (delta**2) * dX(start_pos) / 2 + delta * unscaled_brownian

        alpha = np.exp(-X(proposed) + K(unscaled_brownian) + X(start_pos) -
                       K(unscaled_brownian - delta / 2 * (dX(start_pos) + dX(proposed))) )
        a += min(1, alpha)
        next_state = proposed if stats.bernoulli.rvs(min(1, alpha), size=1) else start_pos
        result = np.append(result, [next_state], axis=0)
    print(a / num_samples)
    return result[::step]

In [3]:
class Ising:
    def __init__(self, weights, biases):
        assert type(weights) == np.ndarray
        assert type(biases) == np.ndarray
        assert biases.ndim == 1, 'biases should be a 1d array'
        assert weights.ndim == 2 and weights.shape[0] == weights.shape[1], 'weights should be square'
        assert np.all(weights == weights.T), 'weights should be symmetric'
        assert weights.shape[1] == biases.shape[0], 'biases and weights have mismatched dimension'

        self.num_vertices = biases.shape[0]
        self.J = weights
        self.bern_J = 4*J

        self.biases = biases
        self.bern_biases = 2*biases - weights @ np.ones(self.num_vertices)
        
    
    def ising_energy(self, state):
        return -(state.T @ self.J @ state) / 2 - np.dot(self.biases, state)


def read_maxcut(fname):
    with open(fname, 'r') as f:
        num_vertices, num_edges = map(int, next(f).strip().split('\t'))
        weights = np.zeros((num_vertices, num_vertices))
#         print(num_edges, num_vertices)
        edges = DictReader(f, fieldnames=['start', 'end', 'weight'], delimiter='\t')
        for edge in edges:
            i, j, w = int(edge['start']) - 1, int(edge['end']) - 1, float(edge['weight'])
            weights[i, j], weights[j, i] = w, w
        return Ising(-weights, np.zeros(num_vertices))


In [13]:
def continuous_relaxation(ising, A, num_samples=1000, epsilon=0.5):
    assert type(A) == np.ndarray
    assert A.ndim == 2 and A.shape[0] == A.shape[1], 'A should be square'
    assert A.shape[1] == ising.num_vertices, 'A and weights have mismatched dimension'

    eigs = np.linalg.eigvalsh(ising.bern_J)
    D = (-eigs[0] + epsilon if eigs[0] < 0 else 0) * np.ones(ising.num_vertices)
    print(eigs, D)
    D = np.diag(D)
    W_D = ising.bern_J + D
    print('eigs of W_D: ', np.linalg.eigvalsh(W_D))
    
    # recall negative log-density
    A_i = np.linalg.inv(A)
    AW_Di = A_i @ np.linalg.inv(W_D) @ A_i.T
    print('AW_Di: ', AW_Di)
    def U(x, A_i=A_i, AW_Di=AW_Di, D=np.diag(D)):
        return (x.T @ AW_Di @ x) / 2 - np.sum(np.log1p(np.exp(A_i @ x + ising.bern_biases - D/2)))
    def dU(x, A_i=A_i, AW_Di=AW_Di, D=np.diag(D)):
        # assume A=I
        temp = np.exp(x + ising.bern_biases - D / 2)
        return x.T @ AW_Di - np.divide(temp, temp+1).T
#     samples = hamiltonian_mc(lambda: np.array([0,0,1], dtype=np.float64), num_samples, U, dU, L=20, delta=1.4)
    samples = mala(lambda: np.array([0,0,0], dtype=np.float64), num_samples, U, dU)
    print(np.mean(samples), np.var(samples, axis=0))

    #MC of samples to get s then x
    print()
    results = []
    for x in samples:
        probs = np.reciprocal(1 + np.exp(-x - ising.bern_biases + np.diag(D) / 2)) #  sigmoid:1 / (1+e^-z)
        sample = [stats.bernoulli.rvs(y) for y in probs]
        print(x, probs, sample)
        results.append(sample)
    
    t = np.array([], dtype=np.float64)
    for s in results:
        t = np.append(mv_norm.rvs(mean=W_D @ s, cov=W_D))
    
    return np.array(results)

In [16]:
num_samples = 20000

J = 0.125 * np.array([[0, -1, 2], [-1, 0, 2], [2, 2, 0]], dtype=np.float64)
biases = 0.125 * np.array([1, 1, -2], dtype=np.float64)
ising = Ising(J, biases)

samples = continuous_relaxation(ising, np.eye(ising.num_vertices), num_samples)
uniques = list(np.unique(samples, return_counts=True, axis=0))
uniques[1] = uniques[1] / 20000
uniques = [x for x in zip(uniques[0], uniques[1])]
print(uniques)

[-1.68614066  0.5         1.18614066] [2.18614066 2.18614066 2.18614066]
eigs of W_D:  [0.5        2.68614066 3.37228132]
AW_Di:  [[ 0.83440824  0.46212692 -0.59307033]
 [ 0.46212692  0.83440824 -0.59307033]
 [-0.59307033 -0.59307033  1.        ]]
0.6791608051857264
1.790600528498679 [3.7639472  3.75261938 4.25039463]

[0. 0. 0.] [0.27526529 0.27526529 0.10977218] [0, 0, 0]
[ 1.08537107 -0.55357354 -0.03375112] [0.52929161 0.17921958 0.10651712] [1, 0, 0]
[ 1.22658799 -1.25647149  1.57586616] [0.56427186 0.09756817 0.37350622] [0, 0, 0]
[ 4.23627679 -1.39596085  1.32185481] [0.96332185 0.08595694 0.31621622] [1, 0, 1]
[ 4.23627679 -1.39596085  1.32185481] [0.96332185 0.08595694 0.31621622] [1, 0, 0]
[ 4.23627679 -1.39596085  1.32185481] [0.96332185 0.08595694 0.31621622] [1, 0, 0]
[4.34223498 0.12357204 3.62218268] [0.96688729 0.30058824 0.8218764 ] [1, 1, 1]
[ 3.76552166 -0.64530044  2.49142907] [0.94253794 0.16612115 0.59829327] [1, 0, 0]
[ 3.76552166 -0.64530044  2.49142907] [0.9425

[1.99306065 2.1061651  4.5252211 ] [0.73594351 0.75732966 0.91924633] [1, 1, 1]
[1.65955386 2.59411947 4.11561348] [0.66629686 0.83562769 0.88314372] [1, 1, 1]
[2.16976533 3.35356318 4.22241916] [0.76882618 0.91571435 0.89372317] [1, 1, 1]
[2.61865534 2.51962151 3.44872038] [0.8389701  0.82513766 0.79505181] [0, 1, 1]
[2.61865534 2.51962151 3.44872038] [0.8389701  0.82513766 0.79505181] [0, 1, 1]
[4.49036965 2.7645576  6.54654739] [0.97131564 0.85772079 0.98849586] [1, 1, 1]
[4.49036965 2.7645576  6.54654739] [0.97131564 0.85772079 0.98849586] [1, 1, 1]
[4.49036965 2.7645576  6.54654739] [0.97131564 0.85772079 0.98849586] [1, 1, 1]
[4.49036965 2.7645576  6.54654739] [0.97131564 0.85772079 0.98849586] [1, 1, 1]
[4.10831034 2.16999366 6.57147014] [0.95852242 0.76886676 0.98877585] [1, 1, 1]
[3.48006674 2.12407243 3.68105539] [0.92497855 0.76060552 0.83033243] [1, 1, 0]
[3.48006674 2.12407243 3.68105539] [0.92497855 0.76060552 0.83033243] [1, 1, 1]
[1.50337617 1.27574391 1.80367434] [0.63

[ 4.08687836 -2.34217058  0.56510516] [0.95766193 0.03522153 0.1782916 ] [1, 0, 0]
[ 3.83797535 -1.22021807  1.1110089 ] [0.94633853 0.10080714 0.27248294] [1, 0, 0]
[3.49024468 0.51509975 5.2060044 ] [0.92568178 0.38865472 0.95742312] [1, 0, 1]
[ 3.33544373 -0.22947957  3.21396027] [0.91430529 0.23191136 0.75415375] [1, 0, 0]
[ 3.33982566 -0.60108254  3.37805592] [0.91464799 0.17233719 0.78329724] [1, 0, 0]
[ 3.33982566 -0.60108254  3.37805592] [0.91464799 0.17233719 0.78329724] [1, 0, 1]
[ 4.21626054 -1.71598241  2.66488979] [0.96260803 0.06392095 0.6391829 ] [1, 0, 1]
[ 4.42148125 -0.87186822  4.43634024] [0.9693327  0.13705856 0.9123978 ] [1, 0, 1]
[ 5.08542301 -1.74976543  3.53508595] [0.98397346 0.06192908 0.80876659] [1, 1, 1]
[ 5.08542301 -1.74976543  3.53508595] [0.98397346 0.06192908 0.80876659] [1, 0, 1]
[ 5.08542301 -1.74976543  3.53508595] [0.98397346 0.06192908 0.80876659] [1, 0, 0]
[ 3.96196119 -0.60630536  3.1536969 ] [0.95229737 0.1715935  0.74281027] [1, 0, 1]
[ 3.961

[1.11159082 2.08875397 2.25786962] [0.53581866 0.7541155  0.54110683] [1, 1, 0]
[1.11159082 2.08875397 2.25786962] [0.53581866 0.7541155  0.54110683] [1, 1, 0]
[2.22066274 2.07714358 2.40065431] [0.7777483  0.7519563  0.57629543] [1, 1, 1]
[1.68709284 3.0083231  3.15456541] [0.67239173 0.884959   0.74297615] [1, 0, 1]
[3.01737808 2.7472502  3.17149923] [0.88587765 0.85559557 0.74619655] [1, 0, 1]
[2.79971495 3.25537977 2.77975059] [0.86195753 0.90782054 0.66522803] [0, 1, 1]
[2.79971495 3.25537977 2.77975059] [0.86195753 0.90782054 0.66522803] [1, 1, 1]
[-1.5883696   5.16308688  1.87594839] [0.07199503 0.98515325 0.44593176] [0, 1, 0]
[-1.03448691  4.297771    3.78820965] [0.11893469 0.96543378 0.84489884] [0, 1, 1]
[-1.10044951  4.49132259  2.15146516] [0.11219439 0.97134217 0.51459456] [0, 1, 0]
[-2.53855046  4.92730873  1.38131377] [0.02912444 0.9812795  0.32921082] [0, 1, 0]
[-2.53855046  4.92730873  1.38131377] [0.02912444 0.9812795  0.32921082] [0, 1, 0]
[-2.53855046  4.92730873 

[1.68672928 3.05101831 1.88248218] [0.67231164 0.88923473 0.44754667] [1, 1, 1]
[2.14869692 2.85874804 2.70957932] [0.76506045 0.86883278 0.64942416] [1, 1, 1]
[2.14869692 2.85874804 2.70957932] [0.76506045 0.86883278 0.64942416] [0, 1, 1]
[2.44643894 1.58198911 1.25182596] [0.81432604 0.64883421 0.30127277] [0, 1, 1]
[-0.77680399  1.17398022  2.61535254] [0.14869486 0.55129636 0.62768127] [0, 1, 1]
[-1.60509637  2.78070254  1.09347627] [0.07088546 0.8596797  0.26902124] [0, 1, 1]
[-1.60509637  2.78070254  1.09347627] [0.07088546 0.8596797  0.26902124] [0, 1, 1]
[-0.94100198  4.11952801  1.33457109] [0.12908511 0.95896612 0.31897219] [0, 1, 1]
[-1.75654173  1.37673392 -0.18082038] [0.06153658 0.60076739 0.09330853] [0, 0, 0]
[-2.99165658  1.11371207  0.10796235] [0.01871152 0.53634621 0.12077538] [0, 0, 0]
[-2.56641875  3.07337975 -0.83057359] [0.02834668 0.89141816 0.05099706] [0, 1, 0]
[-0.64581678  1.59569025  1.07247437] [0.16604964 0.65194959 0.26491133] [1, 0, 1]
[-1.05919608  1.

...

[3.557507   0.63681341 1.40709549] [0.93017864 0.41793483 0.33492909] [1, 1, 0]
[3.557507   0.63681341 1.40709549] [0.93017864 0.41793483 0.33492909] [0, 0, 0]
[ 2.7955243  -0.82850623  0.7805327 ] [0.86145814 0.14226831 0.21206252] [1, 0, 1]
[ 2.7955243  -0.82850623  0.7805327 ] [0.86145814 0.14226831 0.21206252] [1, 0, 0]
[1.2770834  0.02890147 2.00212395] [0.57664434 0.28106826 0.47727906] [1, 1, 0]
[1.2770834  0.02890147 2.00212395] [0.57664434 0.28106826 0.47727906] [0, 1, 0]
[1.19394417 1.12498793 2.56276125] [0.5562296 0.5391491 0.6153106] [0, 1, 1]
[ 2.39894225 -0.29562478  0.89185992] [0.80703713 0.22033845 0.23125996] [1, 0, 0]
[ 2.13108241 -0.79359711  0.49178285] [0.7618796  0.14658163 0.16780175] [1, 1, 0]
[ 3.71620233 -1.90512543  1.80561268] [0.93980777 0.05349461 0.42862639] [1, 0, 1]
[ 3.71620233 -1.90512543  1.80561268] [0.93980777 0.05349461 0.42862639] [1, 1, 0]
[ 3.71620233 -1.90512543  1.80561268] [0.93980777 0.05349461 0.42862639] [1, 0, 1]
[ 4.40609272 -1.996163

[4.43557566 0.59900266 5.15718346] [0.96974892 0.40876632 0.95538793] [1, 1, 1]
[3.42433736 0.55399509 5.21748556] [0.92101854 0.39793536 0.95788869] [1, 1, 1]
[5.08010518 2.9735964  4.08242409] [0.98388938 0.88137606 0.87967475] [1, 1, 0]
[3.82737021 1.4198888  6.52876202] [0.94579742 0.6110715  0.98829184] [1, 1, 1]
[5.15656286 1.42122263 6.21129088] [0.98505753 0.61138846 0.98398714] [1, 1, 1]
[5.15656286 1.42122263 6.21129088] [0.98505753 0.61138846 0.98398714] [1, 0, 1]
[4.77715345 1.87600216 5.04316288] [0.97831229 0.71257677 0.95026786] [1, 1, 1]
[4.67087191 1.23879112 5.08807208] [0.97593885 0.56726985 0.95234781] [1, 0, 1]
[4.67087191 1.23879112 5.08807208] [0.97593885 0.56726985 0.95234781] [1, 0, 1]
[4.67087191 1.23879112 5.08807208] [0.97593885 0.56726985 0.95234781] [1, 0, 1]
[4.67087191 1.23879112 5.08807208] [0.97593885 0.56726985 0.95234781] [1, 0, 1]
[4.02408913 1.57243532 4.97228142] [0.95504167 0.64665431 0.94680915] [1, 0, 1]
[4.02408913 1.57243532 4.97228142] [0.95

[4.09720641 3.03797932 4.62529938] [0.95807871 0.88794391 0.92637054] [1, 1, 1]
[2.2588676  2.91685122 4.35622525] [0.78428211 0.87531365 0.90577923] [1, 1, 1]
[1.83405337 1.29943708 4.77324946] [0.70390916 0.58209189 0.93584688] [1, 1, 1]
[1.76433496 4.87321168 3.79876458] [0.68917488 0.98025943 0.84627697] [1, 1, 1]
[1.24623466 4.1648533  5.81612676] [0.56909613 0.96071303 0.97640992] [1, 1, 1]
[4.15551058 5.25858418 6.42322214] [0.96035889 0.98648721 0.98700553] [1, 1, 1]
[5.33851349 3.85841431 7.2929387 ] [0.98751228 0.94736704 0.99451298] [0, 1, 1]
[4.95160722 4.49417379 7.8787173 ] [0.98172069 0.97142144 0.99693808] [1, 1, 1]
[4.63444718 4.54782794 7.45257415] [0.97506853 0.97287389 0.99531878] [1, 1, 1]
[4.63444718 4.54782794 7.45257415] [0.97506853 0.97287389 0.99531878] [1, 1, 1]
[3.70712735 4.24201164 5.8866944 ] [0.93929235 0.96352394 0.97798185] [1, 1, 1]
[4.45901844 2.43417751 4.77152019] [0.97042912 0.81246498 0.93574298] [1, 1, 1]
[3.24545898 2.71741459 4.59985394] [0.90

[3.40825083 0.80382534 4.83129105] [0.9198404  0.45903081 0.93924464] [1, 1, 1]
[4.8182187  0.34120111 3.43684185] [0.97916668 0.34822077 0.79310948] [1, 0, 0]
[5.45984153 1.96410849 5.24156182] [0.98892328 0.73027892 0.95884924] [1, 1, 1]
[6.0223158  0.7939534  5.63030905] [0.99365829 0.45658041 0.97172895] [1, 0, 1]
[5.26248261 1.62317874 5.91435302] [0.98653908 0.65816071 0.97856963] [1, 0, 1]
[5.26248261 1.62317874 5.91435302] [0.98653908 0.65816071 0.97856963] [1, 1, 1]
[7.1316583  0.29399626 5.88184002] [0.99789973 0.33758519 0.97787708] [1, 0, 1]
[4.75439997 1.65036658 6.01708601] [0.97782423 0.664251   0.98062137] [1, 1, 1]
[5.12844269 0.35041736 5.33779256] [0.98463793 0.35031543 0.962483  ] [1, 0, 1]
[4.36387596 0.73717474 5.75218619] [0.96757319 0.44253119 0.9748914 ] [1, 0, 1]
[5.66304965 1.22862367 3.84136697] [0.99094175 0.56477231 0.85173783] [1, 0, 1]
[5.66304965 1.22862367 3.84136697] [0.99094175 0.56477231 0.85173783] [1, 1, 1]
[4.52291257 1.61360411 3.53919067] [0.97

[3.57584886 2.38727427 3.44525467] [0.93136052 0.80521359 0.79448651] [1, 1, 0]
[3.57584886 2.38727427 3.44525467] [0.93136052 0.80521359 0.79448651] [1, 1, 1]
[1.3960118  3.64292508 3.98277472] [0.605382   0.93552647 0.86872182] [0, 1, 0]
[1.3960118  3.64292508 3.98277472] [0.605382   0.93552647 0.86872182] [0, 1, 1]
[-0.38029301  3.66016027  2.44945462] [0.20613808 0.93655827 0.58816489] [0, 1, 1]
[-0.38029301  3.66016027  2.44945462] [0.20613808 0.93655827 0.58816489] [0, 1, 0]
[-0.38029301  3.66016027  2.44945462] [0.20613808 0.93655827 0.58816489] [0, 1, 1]
[-0.38029301  3.66016027  2.44945462] [0.20613808 0.93655827 0.58816489] [1, 1, 0]
[1.52877173 3.20474895 3.63037685] [0.63661481 0.90349525 0.82307283] [0, 1, 1]
[0.18294727 3.08117792 3.01664258] [0.3132168  0.89217065 0.71576942] [0, 1, 0]
[-0.3140231   2.37836105  3.53476157] [0.21719409 0.80381179 0.80871641] [1, 0, 1]
[2.41246011 1.91935908 1.97897415] [0.80913351 0.72137428 0.47150686] [1, 0, 0]
[0.53398803 1.97853368 2.

In [ ]:
A = np.array([[1, 2, 3], [2, 5, 6], [3, 6, 9]])
Lambda, P = np.linalg.eigh(A)
print(Lambda)
print(np.linalg.eigvalsh(P.T @ np.diag(Lambda) @ P))
print()

print(P.T)
print(P.T @ np.diag([2, 1, 1]))
print(np.diag([2, 1, 1]) @ P)
print(P.T @ P)
print(P.T @ np.diag([4, 1, 1]) @ P)

print()
print(np.linalg.eigvalsh(A + np.diag([1, 1, -10]) @ P))
# print(np.diag([2, 1, 1]) @ A.T)
# print(A @ np.diag([2, 1, 1]))

In [ ]:
file_dir = 'MaxCUT/Hamerly2/maxcut/'
N, prob_id = 10, 0
fname = f'{file_dir}N{N:03}-id{prob_id:02}.txt'
ising = read_maxcut(fname)


In [ ]:
global model
model = CutRBM(fname)

init_func = lambda: np.ones(10)
U = lambda q: -model.prob(torch.Tensor(q), log=True)
random_state = np.ones(10)
print(model.prob(torch.Tensor(random_state)))